In [1]:
!pip install modal

     -------------------------------------- 481.0/481.0 kB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 47.2/47.2 kB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 61.3/61.3 kB 1.6 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     -------------------------------------- 282.1/282.1 kB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 92.0/92.0 kB 2.6 MB/s eta 0:00:00
     -------------------------------------- 370.7/370.7 kB 3.3 MB/s eta 0:00:00
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
     ---------------------------------------- 57.5/57.5 kB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 76.4/76.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.5.3 requires typer<0.8.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!modal setup

⠋ Waiting for authentication in the web browser
⠙ Waiting for authentication in the web browser
⠹ Waiting for authentication in the web browser
The web browser should have opened for you to authenticate and get an API 
token.
If it didn't, please copy this URL into your web browser manually:

⠹ Waiting for authentication in the web browser
https://modal.com/token-flow/tf-06EejetzsgQRCZrBjxuzaW

⠹ Waiting for authentication in the web browser
⠹ Waiting for authentication in the web browser

⠋ Waiting for token flow to complete...
⠋ Waiting for token flow to complete...
⠙ Waiting for token flow to complete...
⠹ Waiting for token flow to complete...
⠼ Waiting for token flow to complete...
⠴ Waiting for token flow to complete...
⠦ Waiting for token flow to complete...
⠧ Waiting for token flow to complete...
⠇ Waiting for token flow to complete...
⠏ Waiting for token flow to complete...
⠋ Waiting for token flow to complete...
⠙ Waiting for token flow to complete...
⠹ Waiting for token flow 

In [5]:
!modal volume create easy-venue

Created volume 'easy-venue' in environment 'None'. 

Code example:

                                                                               
@app.function(volumes={"/my_vol": modal.Volume.from_name("easy-venue")})       
def some_func():                                                               
    os.listdir("/my_vol")                                                      
                                                                               


In [6]:
!modal volume put easy-venue Bangalore_venues.csv Bangalore_venues.csv

- Uploading file 'Bangalore_venues.csv' to 'Bangalore_venues.csv'...
\ Uploading file 'Bangalore_venues.csv' to 'Bangalore_venues.csv'...
/ Uploading file 'Bangalore_venues.csv' to 'Bangalore_venues.csv'...
\ Uploading file 'Bangalore_venues.csv' to 'Bangalore_venues.csv'...
/ Uploading file 'Bangalore_venues.csv' to 'Bangalore_venues.csv'...
\ Uploading file 'Bangalore_venues.csv' to 'Bangalore_venues.csv'...
/ Uploading file 'Bangalore_venues.csv' to 'Bangalore_venues.csv'...
\ Uploading file 'Bangalore_venues.csv' to 'Bangalore_venues.csv'...
/ Uploading file 'Bangalore_venues.csv' to 'Bangalore_venues.csv'...
\ Uploading file 'Bangalore_venues.csv' to 'Bangalore_venues.csv'...
/ Uploading file 'Bangalore_venues.csv' to 'Bangalore_venues.csv'...
\ Uploading file 'Bangalore_venues.csv' to 'Bangalore_venues.csv'...
/ Uploading file 'Bangalore_venues.csv' to 'Bangalore_venues.csv'...
/ Uploading file 'Bangalore_venues.csv' to 'Bangalore_venues.csv'...
✓ Uploaded file 'Bangalore_venues.

In [18]:
import modal

vol = modal.Volume.from_name("easy-venue")
image = (
    modal.Image.debian_slim(python_version="3.10")
    .pip_install("pandas", "numpy", "scikit-learn")
)
app = modal.App(name="easy-venue-recommendation-system",volumes={"/easy-venue": vol},image=image)

def get_recommendations(name, top_n=10):
    import pandas as pd
    from sklearn.feature_extraction.text import TfidfVectorizer
    import numpy as np
    from sklearn.metrics.pairwise import cosine_similarity
    data = pd.read_csv('/easy-venue/Bangalore_venues.csv')
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf.fit(data['categories'])
    text_vec = tfidf.transform([name])
    cosine_sim = cosine_similarity(text_vec, tfidf.transform(data['categories']))
    idxs = np.argsort(cosine_sim[0])[-top_n:][::-1]
    return data.iloc[idxs]['id']

@app.function(image=image, volumes={"/easy-venue": vol})
@modal.web_endpoint(method="POST")
def recommend(category: str)->str:
    ids = get_recommendations(category)
    return ids.to_json()



In [24]:
!modal deploy easy-venue.py

- Creating objects...
\ Creating objects...
/ Creating objects...
\ Creating objects...
└── - Creating mount g:\Codes\NLP\easy-venue.py: Uploaded 0/1 files
/ Creating objects...
└── \ Creating mount g:\Codes\NLP\easy-venue.py: Uploaded 0/1 files
\ Creating objects...
└── / Creating mount g:\Codes\NLP\easy-venue.py: Uploaded 0/1 files
/ Creating objects...
└── \ Creating mount g:\Codes\NLP\easy-venue.py: Uploaded 0/1 files
\ Creating objects...
└── / Creating mount g:\Codes\NLP\easy-venue.py: Uploaded 0/1 files
/ Creating objects...
└── \ Creating mount g:\Codes\NLP\easy-venue.py: Finalizing index of 1 files
\ Creating objects...
└── / Creating mount g:\Codes\NLP\easy-venue.py: Finalizing index of 1 files
/ Creating objects...
├── 🔨 Created mount g:\Codes\NLP\easy-venue.py
└── - Creating function WebApp.recommend...
\ Creating objects...
├── 🔨 Created mount g:\Codes\NLP\easy-venue.py
└── \ Creating function WebApp.recommend...
/ Creating objects...
├── 🔨 Created mount g:\Codes\NLP\easy-

In [30]:
import json
from datetime import datetime
import pandas as pd
import random
import numpy as np

venues_df = pd.read_csv('Bangalore_venues.csv')

# Function to generate random dummy data with specific constraints for price and rating
def generate_custom_dummy_data(row):
    states = ["Karnataka", "Maharashtra", "Tamil Nadu", "Delhi", "West Bengal"]
    countries = ["India", "USA", "UK", "Australia", "Canada"]
    amenities_list = [
        ["Parking", "Wi-Fi", "Restrooms"],
        ["Valet Parking", "Gym", "Spa"],
        ["Swimming Pool", "Bar", "Restaurant"],
        ["Wi-Fi", "Conference Rooms", "Business Center"],
        ["Play Area", "Arcade", "Food Court"]
    ]

    return {
        "venueId": row["id"],
        "name": row["name"],
        "address": str(row["address"]),
        "state": random.choice(states),
        "country": random.choice(countries),
        "zip": f"{random.randint(100000, 999999)}",
        "phone": f"+91 {random.randint(10000, 99999)} {random.randint(10000, 99999)}",
        "email": f"contact{random.randint(1, 100)}@example.com",
        "website": f"https://example.com/{row['id']}",
        "category": row["categories"],
        "price": random.randint(100000, 500000),
        "description": f"{row['name']} is a popular place for {row['categories'].lower()}.",
        "capacity": random.randint(50, 5000),
        "rating": np.round(random.uniform(3.0, 5.0),1) ,
        "latitude": row["lat"],
        "longitude": row["lng"],
        "amenities": random.choice(amenities_list),
        "createdAt": datetime.now().isoformat(),
        "updatedAt": datetime.now().isoformat(),
        "image": f"https://example.com/images/{row['id']}.jpg"
    }

# Transform the DataFrame to JSON format with custom constraints for price and rating
custom_venues_json = [generate_custom_dummy_data(row) for index, row in venues_df.iterrows()]

# Save the custom JSON data to a file
custom_output_path = 'Bangalore_venues_custom.json'
with open(custom_output_path, 'w') as json_file:
    json.dump(custom_venues_json, json_file, indent=2)

custom_output_path



'Bangalore_venues_custom.json'